#### **Download FinBERT model from Hugging Face**

In [ ]:
import os

if os.path.exists(os.path.abspath('./finbert_model/pytorch_model.bin')) == False:
    os.system('git lfs install')
    os.system('git clone https://huggingface.co/ProsusAI/finbert finbert_model')

#### **Add path to access FinBERT library**

In [1]:
import sys

finbert_path = os.path.abspath('./finbert')
if (finbert_path in sys.path) == False:
    sys.path.append(finbert_path)

#### **Download NLTK tokenizer**

In [2]:
import nltk
nltk.download('punkt')

C:\Workspace\80_Framework\anaconda\envs\FinBERT\lib\site-packages\scipy\__init__.py:149: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.3
  UserWarning)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xenos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### **Import libraries**

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import datetime
from finbert.finbert import predict
from transformers import AutoModelForSequenceClassification

#### **Create data output directory**

In [4]:
output_dir = './output'
if os.path.exists(output_dir) == False:
    os.mkdir(output_dir)

#### **Access to Yahoo Finance page**  
Open Yahoo finance page using chrome driver and get main page news contents.  
News contents is wrapped by `container` class, so first get elements using this class info.  
After that, find `a` tag to find link element, and get title and link.  
In last, data will save as csv file.  

#### **TODO**  
Crawl news info from each category. Currently, this crawl main page news only.

In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome(chrome_options)
driver.get('https://finance.yahoo.com/')

elements = driver.find_elements(By.CLASS_NAME, 'container')

news_titles = []
for element in elements:
    # content_card = element.find_elements(By.CLASS_NAME, 'content')
    title_part = element.find_elements(By.TAG_NAME, 'a')
    if len(title_part) == 0:
        continue

    title = title_part[0].get_attribute('title')
    link = title_part[0].get_attribute('href')
    news_titles.append((title, link))

output_path = os.path.abspath('./output/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '.csv')
with open(output_path, '+w', encoding='utf-8') as f:
    f.write('\n'.join([index[0] for index in news_titles]))
    f.close()

print("Main page news loaded.")
print(f"{len(elements)} contents found.")

Main page news loaded.
59 contents found.


In [6]:
articles_index = []

for target_article in news_titles:
    driver.get(target_article[1])
    article_content_area = driver.find_elements(By.CLASS_NAME, 'caas-body')
    if len(article_content_area) == 0:
        continue

    article_content = []
    contents_components = article_content_area[0].find_elements(By.TAG_NAME, 'p')

    for component in contents_components:
        content = component.text
        if len(content) != 0:
            article_content.append(content)
    
    articles_index.append((target_article[0], article_content))
    print("Article parsed : " + target_article[0])

Article parsed : GameStop surges again as meme-stock mania returns
Article parsed : How AMC is set to cash in on the new meme stock rally
Article parsed : US futures tread water ahead of inflation data
Article parsed : Biden targets EVs and chips in sweeping hikes to China tariffs
Article parsed : GameStop's surge is still a long way from 2021's frenzy
Article parsed : Alibaba beats on quarterly revenue, but profit drops 86%
Article parsed : US small business sentiment rebounds in April
Article parsed : Walmart to lay off hundreds of corporate staff: WSJ
Article parsed : The streaming wars are over
Article parsed : Investors are at their 'most bullish' since Nov. 2021: BofA
Article parsed : Waning Fed rate cut bets boost US Treasury yield forecasts: Reuters poll
Article parsed : Hundreds of Syrian refugees head home as anti-refugee sentiment surges in Lebanon
Article parsed : Chevron tops Tesla as most-shorted stock in April, says Hazeltree
Article parsed : Vitol to invest $550 mln in 

In [7]:
import os

article_base_path = os.path.abspath('./output/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '/') + '/'
os.mkdir(article_base_path)

for index in range(0, len(articles_index)):
    article = articles_index[index]
    article_path = article_base_path + str(index) + '.txt'

    with open(article_path, 'w', encoding='utf-8') as f:
        f.write(article[0] + '\n\n')
        f.write('\n'.join([index for index in article[1]]))
        f.close()

In [9]:
model_path = os.path.abspath('./finbert_model')
model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=3, cache_dir=None)

output_path = os.path.abspath('./output/' + datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + '_output/') + '/'
os.mkdir(output_path)
for index in range(0, len(articles_index)):
    article = articles_index[index]
    output_filepath = output_path + str(index) + '.csv'

    print(article[1])
    predict('\n'.join(article[1]), model, write_to_csv=True, path=output_filepath)

05/14/2024 20:59:29 - INFO - root -   Using device: cpu 
05/14/2024 20:59:29 - INFO - finbert.utils -   *** Example ***
05/14/2024 20:59:29 - INFO - finbert.utils -   guid: 0
05/14/2024 20:59:29 - INFO - finbert.utils -   tokens: [CLS] games ##top stock ( gm ##e ) rose as much as 118 % in prem ##ark ##et trading tuesday , adding to a rally monday that sent shares of the video game retailer up 74 % . [SEP]
05/14/2024 20:59:29 - INFO - finbert.utils -   input_ids: 101 2399 14399 4518 1006 13938 2063 1007 3123 2004 2172 2004 12963 1003 1999 26563 17007 3388 6202 9857 1010 5815 2000 1037 8320 6928 2008 2741 6661 1997 1996 2678 2208 20196 2039 6356 1003 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:29 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:29 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

['GameStop stock (GME) rose as much as 118% in premarket trading Tuesday, adding to a rally Monday that sent shares of the video game retailer up 74%.', 'Monday\'s move, in which the stock rose as much as 110% and was halted for volatility several times during the trading session, followed the reemergence of Keith Gill, also known as "Roaring Kitty," whose bull case on GameStop ignited the meme stock rally back in 2021.', "GameStop shares had been flat year to date ahead of Monday's rally, but had risen about 60% in the past month. GameStop stock is now up over 180% over the last month, not including Tuesday's pre-market gains.", "Monday's gain in GameStop was also accompanied by a rally in AMC (AMC), which rose nearly 80%.", 'Early Tuesday, AMC stock was up another 120% in premarket trading. In an SEC filing, AMC also disclosed it issued approximately 72.5 million new shares, raising some $250 million for the company in the process.', "Other heavily-shorted stocks rallying in premarke

05/14/2024 20:59:30 - INFO - finbert.utils -   *** Example ***
05/14/2024 20:59:30 - INFO - finbert.utils -   guid: 0
05/14/2024 20:59:30 - INFO - finbert.utils -   tokens: [CLS] early tuesday , amc stock was up another 120 % in prem ##ark ##et trading . [SEP]
05/14/2024 20:59:30 - INFO - finbert.utils -   input_ids: 101 2220 9857 1010 21962 4518 2001 2039 2178 6036 1003 1999 26563 17007 3388 6202 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:30 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:30 - INFO - finbert.utils -   label: None (id = 9090)
05/14/2024 20:59:30 - INFO - root -   tensor(

['(Bloomberg) -- Retail traders flocking to AMC Entertainment Holdings Inc. in hopes of burning short sellers may also be helping the company shore up its finances.', 'Most Read from Bloomberg', 'Trump Vows ‘Day One’ Executive Order Targeting Offshore Wind', 'Macron Puts French Banks in Play With Plan to Transform Europe', 'Tesla Rehires Some Supercharger Workers Weeks After Musk’s Cuts', 'China to Start $138 Billion Bond Sale on Friday to Boost Economy', 'That’s because the beleaguered movie theater chain, which surged 78% on Monday amid the meme-stock frenzy, has a fundraising deal known as an at-the-market offering that lets it create new shares opportunistically to sell to buyers in the open market. The deal is part of AMC’s long-term effort to secure the capital needed to execute its turnaround, even if that means diluting investors seeking to cash in on short-term rallies.', '“You’d be crazy not to sell shares into this ridiculousness if you’re running the company,” said Tuttle C

05/14/2024 20:59:30 - INFO - finbert.utils -   *** Example ***
05/14/2024 20:59:30 - INFO - finbert.utils -   guid: 0
05/14/2024 20:59:30 - INFO - finbert.utils -   tokens: [CLS] the financing deal has already generated $ 124 million , before commissions and fees , through the sale of 38 . 5 million common shares after being unveiled in march . [SEP]
05/14/2024 20:59:30 - INFO - finbert.utils -   input_ids: 101 1996 12135 3066 2038 2525 7013 1002 13412 2454 1010 2077 13239 1998 9883 1010 2083 1996 5096 1997 4229 1012 1019 2454 2691 6661 2044 2108 11521 1999 2233 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:30 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 20:59:30 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

...

['Nvidia stock investors have a trillion reasons to celebrate. But shockingly owners of two S&P 500 utility stocks have done even better this year.']
['In the realm of retirement savings, only a select few in the United States achieve the milestone of a $5 million nest egg.', 'Data from the Employee Benefit Research Institute, based on the Federal Reserve’s Survey of Consumer Finances, reveals that a mere 0.1% of retirees manage to accumulate over $5 million in their retirement accounts, whereas only 3.2% amass over $1 million.', "Don't Miss:", '82% of Americans aren’t using this government secured 5% passive income stream, are you one of them?', 'Can you guess how many Americans successfully retire with $1,000,000 saved? The percentage may shock you.', 'A look at those with around $5 million in savings, as reported by the Wall Street Journal, shows that such retirees often did not anticipate reaching such financial heights. They were consistent in their savings efforts from early in t

05/14/2024 21:00:13 - INFO - root -   tensor([[ 0.0724, -2.0107,  2.2471],
        [-1.3617,  1.5647,  0.7737],
        [-0.9426, -0.7921,  2.3652],
        [-0.3958, -1.5758,  2.2423],
        [-1.0264,  0.4472,  1.3024]])
05/14/2024 21:00:13 - INFO - finbert.utils -   *** Example ***
05/14/2024 21:00:13 - INFO - finbert.utils -   guid: 0
05/14/2024 21:00:13 - INFO - finbert.utils -   tokens: [CLS] a look at those with around $ 5 million in savings , as reported by the wall street journal , shows that such retire ##es often did not anti ##ci ##pate reaching such financial heights . [SEP]
05/14/2024 21:00:13 - INFO - finbert.utils -   input_ids: 101 1037 2298 2012 2216 2007 2105 1002 1019 2454 1999 10995 1010 2004 2988 2011 1996 2813 2395 3485 1010 3065 2008 2107 11036 2229 2411 2106 2025 3424 6895 17585 4285 2107 3361 7535 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 21:00:13 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

['By Akash Sriram and Casey Hall', "(Reuters) -China's Alibaba Group Holding beat analysts' estimates for fourth-quarter revenue on Tuesday, as a focus on low-cost goods in response to cautious consumer spending helped boost domestic e-commerce sales.", 'Its U.S.-listed shares, however, fell around 3% in premarket trading, as profit fell about 86% in the fourth quarter.', 'The company has had a tumultuous year since announcing the biggest shake-up in its 25-year history in March last year, splitting into six units and refocusing on its core businesses, including domestic e-commerce.', 'Consumers in China have been spending carefully after the COVID-19 pandemic amid an economic slowdown and property slump.', "Alibaba's domestic commerce arm, Taobao and Tmall Group, grew 4% year-over-year with order volume increasing double-digits.", "Analysts expected strong growth from Alibaba's international digital commerce arm, given its investments in building global market share and appetite among

05/14/2024 21:00:14 - INFO - finbert.utils -   tokens: [CLS] analysts expected strong growth from ali ##ba ##ba ' s international digital commerce arm , given its investments in building global market share and appetite among global consumers for low - cost goods from china . [SEP]
05/14/2024 21:00:14 - INFO - finbert.utils -   input_ids: 101 18288 3517 2844 3930 2013 4862 3676 3676 1005 1055 2248 3617 6236 2849 1010 2445 2049 10518 1999 2311 3795 3006 3745 1998 18923 2426 3795 10390 2005 2659 1011 3465 5350 2013 2859 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 21:00:14 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 21:00:14 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
05/14/2024 21:00:14 - INFO - finb

['Alibaba stock slid in early trading Tuesday, after the Chinese e-commerce giant reported a mixed March-quarter.']
